### Environment Setup

In [ ]:
import datetime

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator, img_to_array
from tensorflow.keras.models import Model, load_model, Sequential
from tensorflow.keras.layers import Input, Flatten, Dense, Dropout, GlobalAveragePooling2D, BatchNormalization
from tensorflow.keras.applications import MobileNetV2

%load_ext tensorboard

### Ingest, Image Preprocessing and Augmentation

In [ ]:
train_data_dir = "../../Google Drive/My Drive/498/Project/mushie_image_data/"
num_classes = 2
img_width, img_height = 224, 224
classes = ['poisonous', 'edible']
batch_size = 32

# NOTE: our model will have a single output node
# This means that an output of '0' means a prediction of poisonous,
# And an output of '1' means a prediction of edible
# To flip this, change the order of the classes above

In [ ]:
# Define image augmentation methods here
# As well as the train/validation split (thanks Keras for adding that feature!)
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    validation_split=0.2)

print("Training set:")
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    classes = classes,
    shuffle=True,
    class_mode='binary',
    subset='training')

print("Validation set:")
validation_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    classes = classes,
    class_mode='binary',
    subset='validation')

In [ ]:
### Modeling Setup

In [ ]:
base_model = MobileNetV2(include_top=False, weights='imagenet', input_shape=(img_width, img_height, 3))

In [ ]:
print("These layers will be set to untrainable: ")
for layer in base_model.layers[:-11]:
    layer.trainable = False
    print(layer.name)

In [ ]:
# Now we set the highest convolution block to trainable
# But make sure to not allow any BatchNorm layers be trainable
# https://keras.io/guides/transfer_learning/#finetuning
for layer in base_model.layers[-11:]:
    if "BN" not in layer.name and "bn" not in layer.name:
        layer.trainable = True
        print("\033[93m Trainable: ", layer.name, "\033[0m")
    else:
        layer.trainable = False
        print("Untrainable: ", layer.name)

In [ ]:
# Add in new top layers
# With a sigmoid output node (so we can do binary classification)
x = GlobalAveragePooling2D()(base_model.output)
x = Dense(64, activation='relu')(x)
x = Dropout(0.2)(x)
x = Dense(1, activation='sigmoid')(x)

model = Model(inputs=base_model.input, outputs=x)
model.summary()

### Tensorboard Evaluation

In [ ]:
# To launch tensorboard, run this cell
# Enable auto-reloading in the settings menu (it looks like a gear)
!rm -rf ./logs/ 
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_cb = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

%tensorboard --logdir logs

### Training

In [ ]:
save_checkpoints = tf.keras.callbacks.ModelCheckpoint(
    filepath='./tmp/checkpoint',
    save_weights_only=True,
    monitor='val_acc',
    mode='max',
    save_best_only=True)

In [ ]:
#enable early stopping
es = tf.keras.callbacks.EarlyStopping(
                                 monitor='val_acc',
                                 patience=5,
                                 mode='auto',
                                 baseline=None,
                                 restore_best_weights=True
                                )

In [ ]:
#compile and train the model
model.compile(loss='binary_crossentropy',
              optimizer=tf.keras.optimizers.Adagrad(),
              metrics=['acc', 'AUC'])

In [ ]:
EPOCHS = 10
model.fit(train_generator,
          epochs = EPOCHS,
          steps_per_epoch = train_generator.samples // batch_size,
          validation_data=validation_generator,
          validation_steps = validation_generator.samples // batch_size,
          callbacks=[tensorboard_cb, save_checkpoints]
         )

In [ ]:
model.save('saved_model/mushie_mobilenet_partial.h5')

### Fine-Tuning

In [ ]:
# Unfreeze all the layers to do some fine-tuning
for layer in model.layers:
    # But make sure to not allow any BatchNorm layers be trainable
    # https://keras.io/guides/transfer_learning/#finetuning
    if "BN" not in layer.name and "bn" not in layer.name:
        layer.trainable = True
        print("Trainable: ", layer.name)
    else:
        layer.trainable = False
        print("\033[93m Untrainable: ", layer.name, "\033[0m")

In [ ]:
model.summary()

In [ ]:
# Compile the model to learn at an extremely slow rate
model.compile(loss='binary_crossentropy',
              optimizer=tf.keras.optimizers.Adagrad(1e-5),
              metrics=['acc', 'AUC'])

In [ ]:
EPOCHS = 5
model.fit(train_generator,
          epochs = EPOCHS,
          steps_per_epoch = train_generator.samples // batch_size,
          validation_data=validation_generator,
          validation_steps = validation_generator.samples // batch_size
         )

In [ ]:
model.save('saved_model/mushie_mobilenet_finetuned.h5')